## Sage Data Merge

TO DO:
    - Fix notes / documentation
    - Streamline the "as_is" dataframe, only code for df's in need of modification.   
    

NOTES FOR TEAM:
    - Following 'test_user' records are removed: ULoF3MM1nN gSWn9N D5bzYrfd8E ThpMV2Achc
    
Dataframe Columns Creation:
    
    bart_v4
        - Created columns for baseline, 21-day-assessment, bart250, bart0.25
    
    behavior_choices_4
        - Created columns for baseline, reenrollment
    
    comment_v2 -- DID NOT CLEAN YET!!!
        - CdtW9X Db8hWK duplicate records
    
    delay_discounting
        - Created columns for baseline, 21-day-assessment, dd_time_6_month, dd_money_6_month, dd_money_1_month,           dd_time_1_year    
    
    discounting_raw
        Created columns for baseline, 21-day-assessment, money, probability.
        
    evening_notification
        - Only included "baseline" data.  Did not include "reenrollment" or "set_evening_survey" because of               duplicates.
        
    
        
    
    
    
    - Unexplained duplicates (file, id's):
        - digital-marshmallow-as_a_child_likert_bl-v2_8.8.17.csv    ThpMV2Achc
        - digital-marshmallow-behavior_choices_1_bl-v1_8.8.17.csv   ThpMV2Achc
        - digital-marshmallow-demographics-v2_8.8.17.csv		    ThpMV2Achc
        - digital-marshmallow-behavior_likert_bl-v2_8.8.17.csv		ThpMV2Achc
        - digital-marshmallow-generally_sem_diff_bl-v2_8.8.17.csv	ThpMV2Achc
        - digital-marshmallow-past_year_likert_bl-v3_8.8.17.csv		ThpMV2Achc
        - digital-marshmallow-comments-v2_8.8.17.csv 			    CdtW9X Db8hWK

In [29]:
import os, pandas as pd, numpy as np, re
from functools import reduce
pd.set_option('display.max_columns', 100)

#NOTE TO SELF: redo the relative paths.
sage = '/home/wraikes/Dropbox/partnership/DMTBilly data - Copy/Sage Data'
os.chdir(sage)

In [30]:
def remove_dupes(df):
    #'Must confirm removal of these dupes, especially ThpMV2Achc
    test_users = ['ULoF3MM1nN', 'gSWn9N', 'D5bzYrfd8E', 'ThpMV2Achc']
    return df[~df.externalId.isin(test_users)]

def dupe_check(df):
    return len(df.externalId) == len(df.externalId.unique())

def new_cols(df, append):
    # Remove: 'metadata.json.' and 'data.json.'.
    col_re = re.compile('metadata.json.|data.json.')
    df.columns = list(map(lambda x: re.sub(col_re, '', x), df.columns))
    
    # Append df identifier to columns.
    df.columns = list(map(lambda x: x + append, df.columns))
    df = df.rename(columns = {'externalId' + append: 'externalId'})

    return df

def create_new_df(df, func, attributes):
    dfs = []
    
    for att in attributes:
        new_df = func(df, att[0], att[1])
        new_df = new_cols(new_df, att[2])
        dfs.append(new_df)

    df_merge = reduce(lambda left, right: pd.merge(left, right, how = 'outer',
                                                   on='externalId'), 
                                                   dfs)
    
    return df_merge

In [31]:
test_users = ['ULoF3MM1nN', 'gSWn9N', 'D5bzYrfd8E', 'ThpMV2Achc']
files_to_exclude = ['digital-marshmallow-status_8.8.17.csv',
                    'digital-marshmallow-appVersion_8.8.17.csv']

files_as_is = []
files_to_modify = []

for file in os.listdir():
    if file not in files_to_exclude:
        df = pd.read_csv(file)
        df = remove_dupes(df)
        if dupe_check(df):
            files_as_is.append(file)
        else:
            files_to_modify.append(file)

In [32]:
names_as_is = ['_-_past_year_likert_21', 
               '_-_behavior_choices_1',
               '_-_comments_21',
               '_-_demos',
               '_-_behavior_lk_21',
               '_-_behavior_lk_bl',
               '_-_generally_sem_bl',
               '_-_generally_sem_21',
               '_-_as_a_child',
               '_-_past_year_likert_bl']

In [33]:
new_dfs = {}

for file, name in zip(files_as_is, names_as_is):
    df = pd.read_csv(file)
    df = remove_dupes(df)
    df = new_cols(df, name)
    new_dfs[name] = df

In [45]:
all([dupe_check(df) for name, df in new_dfs.items()])

True

### DataFrame: As_a_child

In [ ]:
as_a_child1 = pd.read_csv('digital-marshmallow-as_a_child_likert_bl-v2_8.8.17.csv')
as_a_child1 = remove_dupes(as_a_child)

dupe_check(as_a_child)

In [ ]:
as_a_child = new_cols(as_a_child, '_-_as_a_child')

### DataFrame: Bart_V4

In [ ]:
bart_v4 = pd.read_csv('digital-marshmallow-bart-v4_8.8.17.csv')
bart_v4 = remove_dupes(bart_v4)

dupe_check(bart_v4)

In [ ]:
def bart_dataframe(df, bart, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['data.json.variable_label'] == bart and row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [ ]:
bart_bl_0 = bart_dataframe(bart_v4, 'BART0.25', 'baseline')
bart_bl_0 = new_cols(bart_bl_0, '_-_bart_v4_bl_0.25')

bart_bl_250 = bart_dataframe(bart_v4, 'BART250.00', 'baseline')
bart_bl_250 = new_cols(bart_bl_250, '_-_bart_v4_bl_250')

bart_21_0 = bart_dataframe(bart_v4, 'BART250.00', '21-day-assessment')
bart_21_0 = new_cols(bart_21_0, '_-_bart_v4_21_0.25')

bart_21_250 = bart_dataframe(bart_v4, 'BART250.00', '21-day-assessment')
bart_21_250 = new_cols(bart_21_250, '_-_bart_v4_21_250')

In [ ]:
dfs = [bart_bl_0, bart_bl_250, bart_21_0, bart_21_250]
bart_v4 = reduce(lambda left, right: pd.merge(left, right, how = 'outer',
                                              on='externalId'), 
                                              dfs)

In [ ]:
dupe_check(bart_v4)

### DataFrame: Behavior_choices_1

In [ ]:
behavior_1 = pd.read_csv('digital-marshmallow-behavior_choices_1_bl-v1_8.8.17.csv')
behavior_1 = remove_dupes(behavior_1)

dupe_check(behavior_1)

In [ ]:
behavior_1 = new_cols(behavior_1, '_-_behavior_1')

### DataFrame: Behavior_choices_4

In [ ]:
behavior_4 = pd.read_csv('digital-marshmallow-behavior_choices_4_bl-v2_8.8.17.csv')
behavior_4 = remove_dupes(behavior_4)

dupe_check(behavior_4)

In [ ]:
def behave_dataframe(df, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [ ]:
#Remove all "reenrollment cases"
behavior_4_bl = behave_dataframe(behavior_4, 'baseline')
behavior_4_bl = new_cols(behavior_4_bl, '_-_behavior_4_bl')

behavior_4_re = behave_dataframe(behavior_4, 'reenrollment')
behavior_4_re = behave_dataframe(behavior_4, '_-_behavior_4_re')

In [ ]:
dfs = [behavior_4_re, behavior_4_bl]
behavior_4 = reduce(lambda left, right: pd.merge(left, right, how = 'outer',
                                                 on='externalId'), 
                                                 dfs)

In [ ]:
dupe_check(behavior_4)

### DataFrame: Behavior_likert_21

In [ ]:
behavior_lk_21 = pd.read_csv('digital-marshmallow-behavior_likert_21-v2_8.8.17.csv')
behavior_lk_21 = remove_dupes(behavior_lk_21)

len(behavior_lk_21.externalId) == len(behavior_lk_21.externalId.unique())

In [ ]:
behavior_lk_21 = new_cols(behavior_lk_21, '_-_behavior_lk_21')

### DataFrame: Behavior_likert_baseline

In [ ]:
behavior_lk_bl = pd.read_csv('digital-marshmallow-behavior_likert_bl-v2_8.8.17.csv')
behavior_lk_bl = remove_dupes(behavior_lk_bl)

dupe_check(behavior_lk_bl)

In [ ]:
behavior_lk_bl = new_cols(behavior_lk_bl, '_-_behavior_lk_bl')

### DataFrame: Comments 21

In [ ]:
comments_21 = pd.read_csv('digital-marshmallow-comments_21-v2_8.8.17.csv')
comments_21 = remove_dupes(comments_21)

dupe_check(comments_21)

In [ ]:
comments_21 = new_cols(comments_21, '_-_comments_21')

### DataFrame: Comments v2 - PLACEHOLDER (INCLUDES EXTRA DUPES)

In [ ]:
comments_v2 = pd.read_csv('digital-marshmallow-comments-v2_8.8.17.csv')
comments_v2 = remove_dupes(comments_v2)

dupe_check(comments_v2)

### DataFrame: Delay Discounting

In [ ]:
delay = pd.read_csv('digital-marshmallow-delay_discounting_raw-v6_8.8.17.csv')
delay = remove_dupes(delay)

dupe_check(delay)

In [ ]:
def delay_dataframe(df, var, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['data.json.variableLabel'] == var and row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [ ]:
bl = 'baseline'
_21 = '21-day-assessment'

delay_attributes = [['dd_time_6_month', bl, '_-_delay_bl_time_6_month'],
                    ['dd_money_6_month', bl, '_-_delay_bl_money_6_month'],
                    ['dd_money_1_month', bl, '_-_delay_bl_money_1_month'],
                    ['dd_time_1_year', bl, '_-_delay_bl_time_1_year'],
                    ['dd_time_6_month', _21, '_-_delay_21_time_6_month'],
                    ['dd_money_6_month', _21, '_-_delay_21_money_6_month'],
                    ['dd_money_1_month', _21, '_-_delay_21_money_1_month'],
                    ['dd_time_1_year', _21, '_-_delay_21_time_1_year']]

delay = create_new_df(df=delay, func=delay_dataframe, attributes=delay_attributes)

In [ ]:
dupe_check(delay)

### DataFrame: Demographics

In [ ]:
demo = pd.read_csv('digital-marshmallow-demographics-v2_8.8.17.csv')
demo = remove_dupes(demo)

dupe_check(demo)

In [ ]:
demo = new_cols(comments_21, '_-_demos')

### DataFrame: Discounting Raw

In [ ]:
discount = pd.read_csv('digital-marshmallow-discounting_raw-v2_8.8.17.csv')
discount = remove_dupes(discount)

dupe_check(discount)

In [ ]:
def discount_dataframe(df, var, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['data.json.variableLabel'] == var and row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [ ]:
bl = 'baseline'
_21 = '21-day-assessment'

discount_attributes = [['pd_constant_money', bl, '_-_discount_bl_money'],
                       ['pd_constant_probabiliy', bl, '_-_discount_bl_prob'],
                       ['pd_constant_money', _21, '_-_discount_21_money'],
                       ['pd_constant_probability', _21, '_-_discount_21_prob']]

discount = create_new_df(df=discount, func=discount_dataframe, attributes=discount_attributes)

In [ ]:
dupe_check(discount)

### DataFrame: Evening Notification

In [ ]:
evening_note = pd.read_csv('digital-marshmallow-evening_notification_time-v2_8.8.17.csv')
evening_note = remove_dupes(evening_note)

dupe_check(evening_note)

In [ ]:
def evening_note_dataframe(df, survey, append):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    new_df = new_cols(new_df, append)
    
    return new_df

In [ ]:
evening_note_bl = evening_note_dataframe(evening_note, 'baseline', '_-_evening_note_bl')

In [ ]:
dupe_check(evening_note_bl)

### DataFrame: Evening Sem - PLACEHOLDER (ONLY INCLUDE PM_SURVEY, NO IDENTIFIERS)

In [ ]:
evening_sem = pd.read_csv('digital-marshmallow-evening_sem_diff-v2_8.8.17.csv')
evening_sem = remove_dupes(evening_sem)

dupe_check(evening_sem)

### DataFrame: Generally Sem Diff 21

In [ ]:
generally_21 = pd.read_csv('digital-marshmallow-generally_sem_diff_21-v2_8.8.17.csv')
generally_21 = remove_dupes(generally_21)

dupe_check(generally_21)

In [ ]:
generally_21 = new_cols(generally_21, '_-_generally_21')

### DataFrame: Generally Sem Diff BL

In [ ]:
generally_bl = pd.read_csv('digital-marshmallow-generally_sem_diff_bl-v2_8.8.17.csv')
generally_bl = remove_dupes(generally_bl)

dupe_check(generally_bl)

In [ ]:
generally_bl = new_cols(generally_bl, '_-_generally_bl')

### DataFrame: GoNoGo - PLACEHOLDER (DUPLICATES: ksJM3Y)

In [ ]:
gonogo = pd.read_csv('digital-marshmallow-goNoGo-v2_8.8.17.csv')
gonogo = remove_dupes(gonogo)

dupe_check(gonogo)

In [ ]:
def gonogo_dataframe(df, var, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['data.json.variable_label'] == var and row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [ ]:
bl = 'baseline'
_21 = '21-day-assessment'

gonogo_attributes = [['go_no_go_stable_stimulus_active_task', bl, '_-_gonogo_bl_stable'],
                     ['go_no_go_variable_stimulus_active_task', bl, '_-_gonogo_21_variable'],
                     ['go_no_go_stable_stimulus_active_task', _21, '_-_gonogo_bl_stable'],
                     ['go_no_go_variable_stimulus_active_task', _21, '_-_gonogo_21_variable']]

gonogo = create_new_df(df=gonogo, func=gonogo_dataframe, attributes=gonogo_attributes)

In [ ]:
dupe_check(gonogo)

In [ ]:
gonogo.externalId[gonogo.externalId.duplicated()]

### DataFrame: Morning Behavior PLACEHOLDER (ONLY INCLUDES AM_SURVEY)

### DataFrame: Morning Notifications

In [ ]:
morning_note = pd.read_csv('digital-marshmallow-morning_notification_time-v3_8.8.17.csv')
morning_note = remove_dupes(morning_note)

dupe_check(morning_note)

In [ ]:
def morning_note_dataframe(df, survey, append):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    new_df = new_cols(new_df, append)
    
    return new_df

In [ ]:
morning_note_bl = morning_note_dataframe(morning_note, 'baseline', '_-_morning_note_bl')

In [ ]:
dupe_check(morning_behave_bl)

### DataFrame: Morning Semantics PLACEHOLDER (ONLY INCLUDES AM_SURVEY)

### DataFrame: Morning Yesterday Likert PLACEHOLDER (ONLY INCLUDES AM_SURVEY)

### DataFrame: Morning Yesterday Sem Diff PLACEHOLDER (ONLY INCLUDES AM_SURVEY)

### DataFrame: PAM Multiple

In [ ]:
pam_mult = pd.read_csv('digital-marshmallow-pam_multiple-v2_8.8.17.csv')
pam_mult = remove_dupes(pam_mult)

dupe_check(pam_mult)

In [ ]:
def pam_mult_dataframe(df, survey, append):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    new_df = new_cols(new_df, append)
    
    return new_df

In [ ]:
pam_mult_bl = pam_mult_dataframe(pam_mult, 'baseline', '_-_pam_mult_bl')
pam_mult_21 = pam_mult_dataframe(pam_mult, '21-day-assessment', '_-_pam_mult_21')

dfs = [pam_mult_bl, pam_mult_21]
pam_mult = reduce(lambda left, right: pd.merge(left, right, how = 'outer',
                                               on='externalId'), 
                                               dfs)

In [ ]:
dupe_check(pam_mult)

### DataFrame: PAM_v2 PLACEHOLDER (ONLY INCLUDES AM_SURVEY AND PM_SURVEY)